<a href="https://colab.research.google.com/github/srilamaiti/ml_works/blob/main/bardapi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install bardapi>=0.1.24
!pip install keybert
!pip install keybert[flair]
!pip install keybert[gensim]
!pip install keybert[spacy]
!pip install keybert[use]
!pip install urllib3

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.6 MB/s eta 0:00:00
  Created wheel for keybert: filename=keybert-0.8.3-py3-none-any.whl size=39124 sha256=97324734a0bcc4659709a57a68d920a8df798f543040dd25978d04caa7ebdc97
  Stored in directory: /root/.cache/pip/wheels/70/88/07/1a3bc11fd1dd5f89924a02dcbca89a3015e25e8faa31f904dc
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=d2f44addecd1b1d4df8cc4c03fbfdd2c5e980db10de1a423818b27a3be332321
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built keybert sentence-transformers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.3/388.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 11.3 MB/s eta 0:00:0

In [13]:
from bardapi import Bard
import bardapi
import os
from bardapi import BardCookies
import requests
from bardapi.constants import SESSION_HEADERS
from keybert import KeyBERT
import transformers
import re
import pandas as pd
import copy

In [3]:
bardapi.__version__

'0.1.38'

In [4]:
kw_model = KeyBERT()

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [14]:
df = pd.read_csv("prod_desc.csv")
orig_df = copy.deepcopy(df)

In [16]:
df = df[['PROD_ID', 'PROD_DESC']]

In [17]:
df.head()

,PROD_ID,PROD_DESC
0,SCMT50325,LOCK & KEY ASSMBLY-50208
1,SCMT50246,TOP CHEST 6 DRAWER
2,SCMT50248,TOP CHEST 8 DRAWER
3,SCMT50207Q,ROLLER CABINET 7 DRAWER
4,SCMT50276GB,11 DRAWER TRIPLE BANK GREEN BLACK


In [18]:
df.tail()

,PROD_ID,PROD_DESC
702934,MXC2200-IN,14 INCH 2200W CHOPSAW
702935,DC122-B5,10.8V CORDLESS DRILL
702936,MXD550-IN,"MAX 13MM, 550W HAMMER DRILL"
702937,MXDD200-IN,MAX 3500W 200MM DIAMOND DRILL
702938,24542SE,CRAFTSMAN 6.0 AMP ANGLE GRINDER- FOB


In [84]:
cookie_dict = {
    "__Secure-1PSID": "dwgMQVFfLmzC5hf8fmc1aNoSoykiAi_-Daubvd9MFd9YZQ0-NC5zyoqpgHYtG5fPeQqzWQ.",
    "__Secure-1PSIDTS": "sidts-CjEBPVxjSmMujK9puKDpPszqQzTpMhWxDwfK3M_z8nkBTwyuSCWmUPtVKphEaHdK3l3kEAA",
    "__Secure-1PSIDCC": "ABTWhQHZRajrSedKyPTU6W_2qWTgp1A15BwRxeYB6Guc0ibTXpGC1ZYVutFE1GrtDt-ydCTKZA0"
}

bard = BardCookies(cookie_dict=cookie_dict)
bard

In [19]:
product_desc = "CRAFTSMAN 6.0 AMP ANGLE GRINDER- FOB"

In [23]:
product_desc = ''.join([i for i in product_desc if not i.isdigit()])
product_desc = re.sub(r'[^\w]', ' ', product_desc)
product_desc = ' '.join(product_desc.split())
print(product_desc)
keywords = kw_model.extract_keywords(product_desc)
print(keywords)
trigram_kw_desc = kw_model.extract_keywords(product_desc, keyphrase_ngram_range=(1, 3), stop_words=None)
top_trigram_kw_desc = trigram_kw_desc[0][0]
print(trigram_kw_desc)
print(top_trigram_kw_desc)

CRAFTSMAN AMP ANGLE GRINDER FOB
[('grinder', 0.5108), ('craftsman', 0.4985), ('amp', 0.3502), ('fob', 0.3298), ('angle', 0.1965)]
[('angle grinder fob', 0.8073), ('amp angle grinder', 0.783), ('craftsman amp angle', 0.7173), ('craftsman amp', 0.6854), ('grinder fob', 0.6736)]
angle grinder fob


In [27]:
input_text = f"classify product category for {top_trigram_kw_desc}"
response = bard.get_answer(input_text)['content']
print(response)

Sure, here is a product category classification for an angle grinder fob:

**Product Category:** Power Tool Safety Accessory

**Specific Product Type:** Angle Grinder Safety Lock

**Subcategory:** Safety Devices

**Additional Category Descriptors:** Protective Accessories

**Key Functions:**

* Prevents accidental starts
* Locks the grinder in the off position
* Provides an extra layer of protection for the user

**Target Audience:**

* Power tool users
* Angle grinder operators
* Individuals concerned about workplace safety

**Reasons for Purchase:**

* To comply with safety regulations
* To reduce the risk of accidents
* To protect the user from injury
* To extend the lifespan of the angle grinder

**Overall Benefits:**

* Angle grinder fobs are a cost-effective way to improve safety.
* They are easy to install and use.
* They can provide peace of mind for the user.
* They can help to prevent costly accidents.

I hope this classification is helpful!


In [28]:
product_desc = "11 DRAWER TRIPLE BANK GREEN BLACK"

In [29]:
product_desc = ''.join([i for i in product_desc if not i.isdigit()])
product_desc = re.sub(r'[^\w]', ' ', product_desc)
product_desc = ' '.join(product_desc.split())
print(product_desc)
keywords = kw_model.extract_keywords(product_desc)
print(keywords)
trigram_kw_desc = kw_model.extract_keywords(product_desc, keyphrase_ngram_range=(1, 3), stop_words=None)
top_trigram_kw_desc = trigram_kw_desc[0][0]
print(trigram_kw_desc)
print(top_trigram_kw_desc)

DRAWER TRIPLE BANK GREEN BLACK
[('drawer', 0.5343), ('black', 0.4395), ('green', 0.4346), ('bank', 0.4058), ('triple', 0.3352)]
[('drawer triple bank', 0.7636), ('bank green black', 0.7065), ('triple bank green', 0.6967), ('drawer triple', 0.6768), ('bank green', 0.5975)]
drawer triple bank


In [30]:
input_text = f"classify product category for {top_trigram_kw_desc}"
response = bard.get_answer(input_text)['content']
print(response)

Sure, here is a product category classification for a drawer triple bank:

* **Product Category:** Drawer Hardware
* **Specific Product Type:** Drawer Slides
* **Subcategory:** Triple Glides
* **Additional Category Descriptors:** Furniture Hardware, Furniture Components

**Key Functions:**

* Enables smooth and effortless drawer operation
* Provides support and stability for heavy drawers
* Offers a versatile and efficient storage solution

**Target Audience:**

* Cabinetry manufacturers
* Furniture makers
* Homeowners
* DIY enthusiasts

**Reasons for Purchase:**

* Enhances the functionality and aesthetics of furniture
* Optimizes storage space and organization
* Supports the weight of heavy items
* Provides a long-lasting and durable solution

**Overall Benefits:**

* Triple drawer slides offer superior functionality and convenience
* They are well-suited for cabinets, storage units, and furniture with heavy drawers
* Triple slides enhance the overall aesthetics and appeal of furnitu

In [38]:
[e for e in response.split("\n") if e.startswith("* **Product Category:**")][0].split("* **Product Category:** ")[1]

'Drawer Hardware'

In [85]:
enhanced_list = []
for e in df.values.tolist()[:5]:
    product_name, orig_product_desc = e[0], e[1]
    product_desc = ''.join([i for i in orig_product_desc if not i.isdigit()])
    product_desc = re.sub(r'[^\w]', ' ', product_desc)
    product_desc = ' '.join(product_desc.split())
    trigram_kw_desc = kw_model.extract_keywords(product_desc, keyphrase_ngram_range=(1, 3), stop_words=None)
    top_trigram_kw_desc = trigram_kw_desc[0][0]
    input_text = f"classify product category for {top_trigram_kw_desc}"
    try:
        response = bard.get_answer(input_text)['content']
        #print(response)
        #print("--------------------------------------------------------------------")
        summary_response = bard.get_answer(f"summarize in 10 words {response}")['content'].split("\n")[-1]
        gist = kw_model.extract_keywords(summary_response.split("\n")[-1], keyphrase_ngram_range=(1, 3), stop_words=None)[0][0]
        print(summary_response)
        print("------------------")
        product_category = [e for e in response.split("\n") if e.startswith("* **Product Category:**")][0].split("* **Product Category:** ")[1]
        specific_product_type = [e for e in response.split("\n") if e.startswith("* **Specific Product Type:**")][0].split("* **Specific Product Type:** ")[1]
        sub_category = [e for e in response.split("\n") if e.startswith("* **Subcategory:**")][0].split("* **Subcategory:** ")[1]
        additional_category_desc = [e for e in response.split("\n") if e.startswith("* **Additional Category Descriptors:**")][0].split("* **Additional Category Descriptors:** ")[1]
        #temp_details = (product_name, orig_product_desc, summary_response, product_category, specific_product_type, sub_category, additional_category_desc)
        temp_details = (product_name, orig_product_desc, summary_response, gist)
        print(temp_details)
    except:
        product_category = None
        specific_product_type = None
        sub_category = None
        additional_category_desc = None
        summary_response = None
        #temp_details = (product_name, orig_product_desc, summary_response, product_category, specific_product_type, sub_category, additional_category_desc)
        temp_details = (product_name, orig_product_desc, summary_response, gist)

    #temp_details = (product_name, orig_product_desc, summary_response, product_category, specific_product_type, sub_category, additional_category_desc)
    #temp_details = (product_name, orig_product_desc, summary_response, gist)
    enhanced_list.append(temp_details)
df_enhanced_prod_desc = pd.DataFrame(enhanced_list)
#df_enhanced_prod_desc.columns = ['product_name', 'orig_product_desc', 'summary', 'product_category', 'specific_product_type', 'sub_category', 'additional_category_desc']
df_enhanced_prod_desc.columns = ['product_name', 'orig_product_desc', 'summary', 'gist']
df_enhanced_prod_desc

Please double-check the cookie values and verify your network environment or google account.
------------------
I hope this is helpful!
------------------
Please double-check the cookie values and verify your network environment or google account.
------------------
Please double-check the cookie values and verify your network environment or google account.
------------------
Please double-check the cookie values and verify your network environment or google account.
------------------


,product_name,orig_product_desc,summary,gist
0,SCMT50325,LOCK & KEY ASSMBLY-50208,None,check the cookie
1,SCMT50246,TOP CHEST 6 DRAWER,None,this is helpful
2,SCMT50248,TOP CHEST 8 DRAWER,None,check the cookie
3,SCMT50207Q,ROLLER CABINET 7 DRAWER,None,check the cookie
4,SCMT50276GB,11 DRAWER TRIPLE BANK GREEN BLACK,None,check the cookie


In [86]:
input_text = f"classify {top_trigram_kw_desc}"
response = bard.get_answer(input_text)['content']
print(response)

Response Error: b')]}\'\n\n38\n[["wrb.fr",null,null,null,null,[8]]]\n57\n[["di",162],["af.httprm",162,"-7457449371632958817",2]]\n25\n[["e",4,null,null,132]]\n'. 
Unable to get response.
Please double-check the cookie values and verify your network environment or google account.


In [75]:
response1 = bard.get_answer(f"summarize in 3 words {response}")['content']
kw_model.extract_keywords(response1.split("\n")[-1], keyphrase_ngram_range=(1, 3), stop_words=None)[0][0]

'functional storage units'